# <p style="background-color:#34495E; color:lightgray; font-size:110%; text-align:center; border-radius:20px 20px;width:950px;height:50px;line-height:50px;border-style:round;">Python DB API</p>

DB-API is an acronym for DataBase Application Programming Interface and a library that lets Python connect to the database server.

## PyODBC
Pyodbc is a Python DB conformant module for ODBC databases. 

**ODBC** (Open Database Connectivity) is the primary native data access API for applications written in C and C++ for SQL Server. Other languages that can use ODBC include COBOL, Perl, PHP, and Python. ODBC is widely used in data integration scenarios.

In [1]:
!pip install pyodbc

In [2]:
import pyodbc

In [3]:
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'SQL Server Native Client 11.0',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']

***

## SQL SERVER CONNECTION

### SQL Server Authentication

In [4]:
def create_server_connection(driver, server, database, user, password):
    conn = None
    try: 
        conn = pyodbc.connect(DRIVER=driver, SERVER=server, DATABASE=database, UID=user, PWD=password)
        print("MS SQL Server Database connection successful!")
    except Exception as err:
        print(f"ERROR:\t'{err}'")
    return conn

In [5]:
driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost'
database = 'master'
user = 'sa'          # 'sa' means system administrator
password = '-----'   # your password

conn = create_server_connection(driver, server, database, user, password)

ERROR:	'('28000', "[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'sa'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'sa'. (18456)")'


In [6]:
 # conn.close()  # to close the sql server connection

### Windows Authentication

In [7]:
conn_string = """driver={ODBC Driver 17 for SQL Server}; 
                 server=localhost; 
                 database=master; 
                 TRUSTED_CONNECTION=yes;"""

In [8]:
conn = pyodbc.connect(conn_string);

In [9]:
conn

***

## Connection Objects

**cursor :** to access the data in the database
    
**commit :** This method commits the current transaction. It allows the transactions made here to be reflected (commit) to the sql server. If you do not commit, they will not be reflected.

**connection.close :** to close the sql server connection

### Cursor process

In [10]:
crs = conn.cursor()

In [11]:
conn.autocommit = True 

In [12]:
#conn.commit() -- for desired transactions

***

## Cursor Objects

### Create Database

In [13]:
crs.execute('CREATE DATABASE TestA')

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Database 'TestA' already exists. Choose a different database name. (1801) (SQLExecDirectW)")

In [ ]:
def create_database(conn, create_db_query):   
    crs = conn.cursor() 
    try:
        crs.execute(create_db_query)  #prepares and executes SQL
        print("Database is Created Successfully")
    except Exception as err:
            print(f"ERROR:\t'{err}'")

In [ ]:
create_db_query = 'CREATE DATABASE TestB'

In [ ]:
create_database(conn, create_db_query)

In [ ]:
crs.execute('DROP DATABASE TestB')

### Executing a query

All SQL statements are executed using the Cursor execute() function. 

In [ ]:
query = 'USE TestA'

In [ ]:
crs.execute(query)

In [ ]:
def execute_query(conn, query):
    crs = conn.cursor()
    try:
        crs.execute(query)
        conn.commit()
        print("Query Succeessful!")
    except Exception as err:
        print(f"ERROR:\t'{err}'")

In [ ]:
execute_query(conn,query)

### Create Table

In [ ]:
query = """CREATE TABLE TestTable (
                ID INT IDENTITY (1,1) NOT NULL, 
                FirstName VARCHAR(255) NOT NULL,
                LastName VARCHAR(255) NOT NULL,
                PRIMARY KEY(ID))"""

In [ ]:
execute_query(conn, query)

### Insert Values

In [ ]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES ('Bob', 'Marley')")

In [ ]:
# parameterized
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Bruce', 'Lee')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", ('Jerry', 'Blue'))
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", ('Katie', 'Holmes'))

In [ ]:
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Tom', 'Cat')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Jerry', 'Mouse')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Owen', 'William')
crs.execute("INSERT TestTable (FirstName, LastName) VALUES (?,?)", 'Stefan', 'Müller')

In [ ]:
#crs.execute("SELECT * FROM TestTable").fetchall()

### Updating and Deleting with rowcount

In [ ]:
crs.execute("DELETE FROM TestTable WHERE FirstName = 'Katie'")
crs.rowcount

In [ ]:
deleted = crs.execute("DELETE FROM TestTable WHERE FirstName = ?", 'Jerry').rowcount
print(deleted, 'row(s) deleted')

In [ ]:
crs.execute("UPDATE TestTable SET FirstName='Natalie', LastName='Portman' WHERE ID=8")
print(crs.rowcount, 'row(s) updated')

In [ ]:
#crs.execute("SELECT * FROM TestTable").fetchall()

***

## Cursor Functions -- Reading Data

All SQL statements are executed using the Cursor execute() function. 

If the statement returns rows, such as a select statement, you can retrieve them using the Cursor fetch functions - fetchone(), fetchall(), fetchmany(). 

If there are no rows, fetchone() will return None, whereas fetchall() and fetchmany() will both return empty lists.

#### Fetchone

In [ ]:
crs.execute('SELECT FirstName, LastName FROM TestTable')

In [ ]:
row = crs.fetchone()  # it returns the first record of the table
row

In [ ]:
crs.fetchone()  # it returns the next record of the table

The `fetchone()` function returns None when all rows have been retrieved.

In [ ]:
crs.execute('select FirstName, LastName from TestTable')
row = crs.fetchone()

print('LastName:', row[1])         # access by column index (zero-based)
print('LastName:', row.LastName)   # access by name

In [ ]:
crs.execute('select FirstName, LastName from TestTable')
while True:
    row = crs.fetchone()
    if not row:
        break
    print('FirstName:', row.FirstName)

#### Fetchmany

In [ ]:
crs.execute("""SELECT ID, FirstName, LastName 
                FROM TestTable""")
rows = crs.fetchmany(3)
rows

In [ ]:
rows[0]

#### Fetchall

In [ ]:
crs.execute('SELECT ID, FirstName, LastName FROM TestTable')
rows = crs.fetchall()
rows 

In [ ]:
for row in rows:
    print(row.ID, row.FirstName, row.LastName)

***

## Connecting Different Databases

In [ ]:
crs.execute('USE SampleRetail')

In [ ]:
crs.execute("SELECT * FROM product.category").fetchall()

In [ ]:
query = """SELECT  
                RIGHT(email, LEN(email)-CHARINDEX('@', email)) AS DomainType,
                COUNT(RIGHT(email, LEN(email)-CHARINDEX('@', email))) AS NumofDomains
           FROM 
                sale.customer
           GROUP BY
                RIGHT(email, LEN(email)-CHARINDEX('@', email))
           ORDER BY
                NumofDomains DESC;"""

In [ ]:
crs.execute(query).fetchall()

***

## Pandas Methods

### Create Dataframe

In [ ]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
query = 'USE TestA'
execute_query(conn,query)

In [ ]:
crs.execute('SELECT * FROM TestTable')
results = crs.fetchall()
results 

In [ ]:
df = pd.DataFrame(results)

In [ ]:
df

In [ ]:
from_db = []

for result in results:
    result = list(result)
    from_db.append(result)

In [ ]:
from_db

In [ ]:
cols = ['ID','FirstName','LastName']
df = pd.DataFrame(from_db, columns = cols)
df

***

In [ ]:
query = 'USE SampleRetail'
execute_query(conn,query)

In [ ]:
df_prod = pd.read_sql("SELECT * FROM product.product", con=conn)
df_prod.head(10)

In [ ]:
query = """
        CREATE TABLE product_new1 (
            product_id int,
            product_name varchar(255),
            brand_id int,
            category_id int ,
            model_year int,
            list_price decimal(10,2)
        )
        """
execute_query(conn, query)

In [ ]:
df_prod.to_dict('records')

In [ ]:
records = df_prod.to_dict('records')

In [ ]:
sql = 'INSERT product_new1 VALUES(?,?,?,?,?,?)'

In [ ]:
for k,v in enumerate(records, start=1):
    print(k,v)

In [ ]:
for k,v in enumerate(records):
    print(v['product_name'])

In [ ]:
for k,v in enumerate(records):
    crs.execute(sql, (v['product_id'], v['product_name'], v['brand_id'], v['category_id'], 
                      v['model_year'], v['list_price']))

In [ ]:
crs.execute("SELECT * FROM product_new1").fetchall()

In [ ]:
conn.close()